In [ ]:
import requests
from bs4 import BeautifulSoup
from  selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time as time
import getpass
import urllib.request
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymongo
%matplotlib inline

In [ ]:
def select_first(driver):
    first = driver.find_element_by_css_selector('div._9AhH0') 
    #find_element_by_css_selector 함수를 사용해 요소 찾기
    first.click()
    time.sleep(3) #로딩을 위해 3초 대기
#본문 내용, 작성 일시, 위치 정보 및 해시태그(#) 추출
import re 
def get_content(driver):
    # 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')    
    # 2. 본문 내용 가져오기
    try:  			#여러 태그중 첫번째([0]) 태그를 선택  
        content = soup.select('div.C4VMK > span')[0].text 
        			#첫 게시글 본문 내용이 <div class="C4VMK"> 임을 알 수 있다.
                                #태그명이 div, class명이 C4VMK인 태그 아래에 있는 span 태그를 모두 선택.
    except:
        content = ' ' 
    # 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content) # content 변수의 본문 내용 중 #으로 시작하며, #뒤에 연속된 문자(공백이나 #, \ 기호가 아닌 경우)를 모두 찾아 tags 변수에 저장
    # 4. 작성 일자 가져오기
    try:
        date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10] #앞에서부터 10자리 글자
    except:
        date = ''
    # 5. 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1] 
    except:
        like = 0
    # 6. 위치 정보 가져오기
    try:
        place = soup.select('div.JF9hh')[0].text
    except:
        place = ''
    data = [content, date, like, place, tags]
    return data 

def move_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow') 
    right.click()
    time.sleep(3)

In [ ]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75'    

options = webdriver.ChromeOptions()

options.add_argument('user-agent={0}'.format(user_agent))
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument('headless')

driver = webdriver.Chrome("chromedriver.exe", options=options)

driver.get("https://www.instagram.com/accounts/login/")
driver.maximize_window()


username = 'ID입력'
password = 'PW입력'
 
element_id = driver.find_element_by_name("username")
element_id.send_keys(username)
element_password = driver.find_element_by_name("password")
element_password.send_keys(password)
 
time.sleep(3)

driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()

time.sleep(3)

In [ ]:
word = '검색어 입력'

driver.get('https://www.instagram.com/explore/tags/' + word)

time.sleep(3)

In [ ]:
from pymongo import MongoClient
from pymongo.cursor import CursorType

host = "localhost"
port = "27017"

try:
    mongo = MongoClient(host, int(port))
except:
    print("Mongo접속 Error")

In [ ]:
def insert_item_one(mongo, dates, db_name=None, collection_name=None):
    result = mongo[db_name][collection_name].insert_one(dates).inserted_id
    return result

def insert_item_many(mongo, dates, db_name=None, collection_name=None):
    result = mongo[db_name][collection_name].insert_many(dates).inserted_ids
    return result

def find_item_one(mongo, condition=None, db_name=None, collection_name=None):
    result = mongo[db_name][collection_name].find_one(condition, {"_id": False})
    return result

def find_item(mongo, condition=None, db_name=None, collection_name=None):
    result = mongo[db_name][collection_name].find(condition, {"_id":False}, no_cursor_timeout=True, cursor_type=CursorType.EXHAUST)
    return result

In [ ]:
from tqdm import tqdm_notebook

select_first(driver) 
#5. 비어있는 변수(results) 만들기
results = [] 

result_contents =[]
result_hashtag = []
# Dict의 형태로 변환시킨 list
result_li = []
#여러 게시물 크롤링하기
target = 10000 #크롤링할 게시물 수
for i in tqdm_notebook(range(target)):
    data = get_content(driver) #게시물 정보 가져오기
    results.append(data)
    insert_item_one(mongo, {"_id" : i , "cont": "{}".format(results[i][0]), "hash" : "{}".format(results[i][-1])}, "insta", "mope")
    move_next(driver)
    
    
# result_li
# print(results[:2])

In [ ]:
df = pd.DataFrame(results,columns=['content','date','like',' ','tags'])
df

In [ ]:
# insta collection 안에 있는 모든 파일을 find 해라.
cursor = find_item(mongo,None,"insta2","insta2")
for list in cursor:
    print(list)

In [ ]:
df['date'].unique()

In [ ]:
print(html)